In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
model='/content/drive/MyDrive/mistral_model'

In [3]:
%%capture
import torch
major_version, minor_version = torch.cuda.get_device_capability()
# Must install separately since Colab has torch 2.2.1, which breaks packages
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    # Use this for new GPUs like Ampere, Hopper GPUs (RTX 30xx, RTX 40xx, A100, H100, L40)
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    # Use this for older GPUs (V100, Tesla T4, RTX 20xx)
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass

In [12]:
import pandas as pd
from datasets import load_metric
from transformers import AutoTokenizer
from unsloth import FastLanguageModel


In [8]:
max_seq_length = 2048
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True

In [9]:
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = model, # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Mistral patching release 2024.4
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.25.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unused kwargs: ['quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/971 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

Unsloth 2024.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [20]:
# Load the dataset
test_df = pd.read_csv('NLP_Recipe_test.csv')  # Ensure this path is correct

# Load the BLEU metric
bleu_metric = load_metric('bleu')

# Function to generate recipes and compute BLEU scores
def generate_and_score(index, row):
    ner_input = row['ner']  # Key ingredients input
    true_output = row['ingredients'] + " " + row['steps']  # True combined output

    # Prepare the prompt
    recipe_prompt = f"""Given the following key ingredients, generate the full ingredient list with quantities and cooking steps:

    ### Key Ingredients:
    {ner_input}

    ### Full Ingredients and Steps:
    """

    # Tokenize and generate the output
    inputs = tokenizer(
        [recipe_prompt],
        return_tensors="pt"
    ).to('cuda')

    outputs = model.generate(**inputs, max_new_tokens=512, use_cache=True)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract the generated recipe part after "### Full Ingredients and Steps:"
    generated_recipe = generated_text.split("### Full Ingredients and Steps:\n")[1].strip()

    # Compute BLEU score
    reference = [true_output.split()]
    candidate = generated_recipe.split()
    bleu_score = bleu_metric.compute(predictions=[candidate], references=[reference])

    return bleu_score['bleu'], generated_recipe

# Iterate over the first 5 rows and compute BLEU scores
for index, row in test_df.head(5).iterrows():
    score, recipe = generate_and_score(index, row)
    print(f"Row {index} BLEU Score: {score}")
    print("Generated Recipe and Steps:", recipe)
    print("----------------------------------------")

/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for bleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/bleu/bleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Row 0 BLEU Score: 0.1311109144771996
Generated Recipe and Steps: 300.0 gram spaghetti, 400.0 gram kipfilet, 1.0 bag kerstomaten, 1.0 bag basilicum, 1.0 bag margarine, 1.0 bag knorr kruidenpasta spaghetti bolognese kook de spaghetti volgens de aanwijzingen op de verpakking . snijd de kipfilet in stukjes . snijd de kerstomaten in blokjes . snijd de basilicum in stukjes . verhit de margarine in een koekenpan en bak de kipfilet in 2 minuten aan beide kanten . voeg de kerstomaten toe en bak 2 minuten . voeg de basilicum toe en bak 1 minuut . voeg de knorr kruidenpasta spaghetti bolognese toe en bak 1 minuut . schep de spaghetti op de bord en schep de kruidenpasta erover . schep de kruidenpasta erover . schep de kruidenpasta erover . schep de kruidenpasta erover . schep de kruidenpasta erover . schep de kruidenpasta erover . schep de kruidenpasta erover . schep de kruidenpasta erover . schep de kruidenpasta erover . schep de kruidenpasta erover . schep de kruidenpasta erover . schep de kruid

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Row 1 BLEU Score: 0.019156091615339783
Generated Recipe and Steps: 1 whole chicken without giblets, 1 small handful of fresh rosemary sprigs, 1 small handful of fresh thyme sprigs, peel from one small lemon, sliced, 2 garlic cloves, 2 large onions, peeled and sliced, salt and pepper, 2 tablespoons olive oil, 1/4 cup honey, 1/2 cup white wine, 1 cup chicken broth, fresh rosemary and thyme sprigs for garnish optional preheat oven to 350degf . place chicken in a large roasting pan . season with salt and pepper . place rosemary, thyme, lemon peel and garlic in the cavity of the chicken . place onions in the bottom of the roasting pan . drizzle with olive oil and honey . place chicken on top of onions . drizzle chicken with olive oil and honey . pour wine and chicken broth over the chicken . roast for 1 1/2 hours or until chicken is done . remove from oven and let rest for 10 minutes . garnish with fresh rosemary and thyme sprigs, if desired.
----------------------------------------


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Row 2 BLEU Score: 0.10233668153683255
Generated Recipe and Steps: 1.0 pound fresh figs, rinsed and halved, 1.0 cup sugar, 1.0 cup filtered water, 1.0 tablespoon freshly squeezed lemon juice, 1.0 cup fig puree, 1.0 cup honey, 1.0 cup ginger ale, 1.0 cup vodka, 1.0 cup freshly squeezed lemon juice, ice, 1.0 cup fig puree, 1.0 cup honey, 1.0 cup ginger ale, 1.0 cup vodka, 1.0 cup freshly squeezed lemon juice in a medium saucepan, combine the figs, sugar, water and lemon juice . bring to a boil over medium high heat, then reduce the heat to medium low and simmer for 10 minutes . remove from the heat and let cool . puree the figs in a blender or food processor . strain the puree through a fine mesh sieve . discard the solids . combine the fig puree, honey and ginger ale in a large pitcher . add the vodka and lemon juice . stir to combine . refrigerate until ready to serve . to serve, fill a glass with ice . pour the fig martini over the ice . garnish with a fig half, if desired.
-----------

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Row 3 BLEU Score: 0.1804868872130938
Generated Recipe and Steps: 1/2 cup quick guacamole, 1 avocado, 1 tablespoon lime juice, kosher salt, 1/2 cup veggies, 1 tablespoon extra virgin olive oil, 1/2 red, /2 medium white onion, 1/2 teaspoon ground cumin, /2 teaspoon chili powder, pinch of red pepper flakes, kosher salt, 2 cloves garlic, 1 tablespoon fresh lime juice, 4 fajitas and garnishes, 4 corn tortillas, olive oil spray, 2 large eggs, crumbled feta or queso fresco cheese, handful of fresh cilantro, freshly ground black pepper, hot sauce and/or your favorite salsa in a small bowl, combine the guacamole, avocado, lime juice and salt . set aside . in a large bowl, combine the veggies, olive oil, red onion, cumin, chili powder, red pepper flakes and salt . set aside . in a small bowl, combine the garlic, lime juice and salt . set aside . in a large bowl, combine the guacamole, avocado, lime juice and salt . set aside . in a large bowl, combine the veggies, olive oil, red onion, cumin, ch